In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st


In [8]:
# Read the mouse data and the study results:
mouse_metadata_path = pd.read_csv("data/Mouse_metadata.csv")
#mouse_metadata
study_results_path = pd.read_csv("data/Study_results.csv")





In [10]:
# Combine the data into a single dataset:
mice_results = pd.merge(mouse_metadata, study_results, on = "Mouse ID")
mice_results_df = mice_results
mice_results_df.head()

# Display the data table for preview
mice_results_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [14]:
# Checking the number of mice.
mouse_total = mice_results_df["Mouse ID"].nunique()
mouse_total

249